In [186]:
from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa
import pandas as pd
import requests
from math import ceil
import os

In [132]:
# Connect to Yahoo API
sc = OAuth2(None, None, from_file='oauth2.json')

[2024-11-26 15:20:40,253 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2024-11-26 15:20:40,311 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 4525.374401807785
[2024-11-26 15:20:40,311 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2024-11-26 15:20:40,318 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [134]:
# get game object
gm = yfa.Game(sc, 'nhl')

leagues = gm.league_ids(year = 2024)

# Get the league object
lg = gm.to_league('453.l.92034')

# Get the team object
team = lg.to_team(lg.team_key())

In [136]:
# Get team roster
roster = team.roster()

# Get free agents
fa = lg.free_agents('Util')
goalie_fa = lg.free_agents('G')

# Get taken players
tp = lg.taken_players()

In [194]:
# Create a dataframe for free agents and taken players
tp_df = pd.DataFrame(tp)
fa_df = pd.DataFrame(fa)
goalie_fa_df = pd.DataFrame(goalie_fa)

# Concatenate all the dataframes into a player dataframe
player_df = pd.concat([tp_df,fa_df, goalie_fa_df], ignore_index=True)

In [114]:
# Retrieve player_ids and load to a dataframe
player_ids_df = pd.DataFrame(player_df['player_id'])
firstrow = player_ids_df['player_id'].iloc[0]

# Create a player stats dataframe
player_stats = lg.player_stats(firstrow, 'season')
player_stats_df = pd.DataFrame(player_stats)
player_stats_df.drop(index=player_df.index[0], axis=0, inplace=True)

# Loop through each player id and concatenate the results to the player stats dataframe      
for i in player_ids_df['player_id']:
    new_row = pd.DataFrame(lg.player_stats(i, 'season'))
    player_stats_df = pd.concat([player_stats_df, new_row], ignore_index=True)

player_stats_df

     player_id             name position_type     G     A  PPP  SHP  GWG  \
0         3358      Brent Burns             P   1.0   5.0  0.0  0.0  0.0   
1         3637    Alex Ovechkin             P  15.0  10.0  7.0  0.0  3.0   
2         3638    Evgeni Malkin             P   5.0  15.0  6.0  0.0  1.0   
3         3737    Sidney Crosby             P   8.0  13.0  8.0  0.0  3.0   
4         3788     Anze Kopitar             P   6.0  19.0  9.0  1.0  3.0   
..         ...              ...           ...   ...   ...  ...  ...  ...   
766       8871  Arvid Soderblom             G   NaN   NaN  NaN  NaN  NaN   
767       9232   Jaxson Stauber             G   NaN   NaN  NaN  NaN  NaN   
768      30247     Yaniv Perets             G   NaN   NaN  NaN  NaN  NaN   
769      30487    Erik Portillo             G   NaN   NaN  NaN  NaN  NaN   
770      30774  Aleksei Kolosov             G   NaN   NaN  NaN  NaN  NaN   

      SOG   HIT  ... Off-ZS Def-ZS ZS-Pct GStr Shifts total_points    W    GA  \
0    4

In [142]:
# Create a dataframe of the ownership for the player dataframe
player_own = lg.ownership([firstrow])
player_own_df = pd.DataFrame(player_own)
player_own_df = player_own_df.transpose()
player_own_df = player_own_df.rename_axis('player_id').reset_index()
player_own_df.drop(index=player_df.index[0], axis=0, inplace=True)

# Loop through each player id and concatenate the results to the player stats dataframe      
for i in tp_df['player_id']:
    new_row = pd.DataFrame(lg.ownership([i]))
    new_row = new_row.transpose()
    new_row = new_row.rename_axis('player_id').reset_index()
    player_own_df = pd.concat([player_own_df, new_row], ignore_index=True)

player_own_df

    player_id ownership_type            owner_team_name
0        3358           team         The Juuse is loose
1        3637           team        Nala poops on grass
2        3638           team               Utah Soakers
3        3737           team               Team Lemieux
4        3788           team               Utah Soakers
..        ...            ...                        ...
253     30545           team               Utah Soakers
254     30548           team  Chester Mew's Choice Team
255     30832           team         The Juuse is loose
256     31759           team               Utah Soakers
257     32763           team       No Willie? No Hedman

[258 rows x 3 columns]


In [196]:
# Join player_own_df and player_stats_df
player_own_df['player_id']=player_own_df['player_id'].astype(int)
player_stats_df = player_stats_df.drop(columns=['name', 'position_type'])
player_df = pd.merge(player_df, player_stats_df, on='player_id', how='left')
player_df = pd.merge(player_df, player_own_df, on='player_id', how='left')

player_df

,player_id,name,position_type,eligible_positions,percent_owned,status,G,A,PPP,SHP,...,ZS-Pct,GStr,Shifts,total_points,W,GA,SV,SHO,ownership_type,owner_team_name
0,3358,Brent Burns,P,"[D, Util]",59,,1.0,5.0,0.0,0.0,...,64.73,0.0,543.0,80.3,NaN,NaN,NaN,NaN,team,The Juuse is loose
1,3637,Alex Ovechkin,P,"[LW, RW, Util, IR, IR+]",98,IR,15.0,10.0,7.0,0.0,...,77.03,6.0,360.0,171.7,NaN,NaN,NaN,NaN,team,Nala poops on grass
2,3638,Evgeni Malkin,P,"[C, LW, Util]",69,,5.0,15.0,6.0,0.0,...,75.6,4.0,509.0,126.5,NaN,NaN,NaN,NaN,team,Utah Soakers
3,3737,Sidney Crosby,P,"[C, Util]",100,,8.0,13.0,8.0,0.0,...,66.58,2.0,544.0,156.8,NaN,NaN,NaN,NaN,team,Team Lemieux
4,3788,Anze Kopitar,P,"[C, Util]",80,,6.0,19.0,9.0,1.0,...,58.02,1.0,474.0,115.3,NaN,NaN,NaN,NaN,team,Utah Soakers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,8871,Arvid Soderblom,G,[G],3,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,52.4,1.0,14.0,176.0,0.0,NaN,NaN
767,9232,Jaxson Stauber,G,[G],3,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,-,-,-,-,NaN,NaN
768,30247,Yaniv Perets,G,[G],0,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,-,-,-,-,NaN,NaN
769,30487,Erik Portillo,G,[G],0,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,-,-,-,-,NaN,NaN


In [198]:
# Save dataframe into a csv to build visualizations
output_path = 'vhp_player_data.csv'
player_df.to_csv(output_path, mode='w', index = True)

In [188]:
# Load free agent table and query the Free Agent table to find necessary player id
#fa_df = pd.DataFrame(fa)
#fa_df

#fa_df.loc[fa_df['name'] == 'Pius Suter']

'C:\\Users\\acer\\DataScience\\Fantasy\\FantasyHockey'

In [22]:
# Add and drop players
#sc = OAuth2(None, None, from_file='oauth2.json') # Connect to Yahoo API
#lg = gm.to_league('453.l.92034') # Get league
#team = lg.to_team(lg.team_key()) # Get team
#team.add_and_drop_players(7829, 4967)

[2024-11-13 00:01:41,986 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2024-11-13 00:01:41,993 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1068.8123824596405
[2024-11-13 00:01:41,995 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


RuntimeError: b'<?xml version="1.0" encoding="UTF-8"?>\n<error xml:lang="en-us" yahoo:uri="http://fantasysports.yahooapis.com/fantasy/v2/league/453.l.92034/transactions" xmlns:yahoo="http://www.yahooapis.com/v1/base.rng" xmlns="http://www.yahooapis.com/v1/base.rng">\n <description>Player Jeff Skinner is not on team Chester Mew s Choice Team.</description>\n <detail/>\n</error>'